# **NER(Named Entity Recognition)**

- 參考:<br>
https://huggingface.co/docs/transformers/task_summary#named-entity-recognition<br>
- 本篇用於測試所使用的模型為隨機挑選的

In [1]:
from IPython.display import clear_output

In [2]:
#句子

#英文
sequence_en1 = """Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO,
therefore very close to the Manhattan Bridge which is visible from the window."""

#中文
#Davlan/bert-base-multilingual-cased-ner-hrl(model)的訓練資料集中的其中一筆資料.
sequence_zh1='我们藏有一册１９４５年６月油印的《北京文物保存保管状态之调查报告》，调查范围涉及故宫、历博、古研所、北大清华图书馆、北图、日伪资料库等二十几家，言及文物二十万件以上，洋洋三万余言，是珍贵的北京史料。'
#任意挑選的文本
sequence_zh2='马英九，台湾政治人物、学者，曾任第12、13任中华民国总统、第4、6任中国国民党主席等职。祖籍江西省永新县，籍贯湖南省湘潭县，马鹤凌和秦厚修之子，生于英属香港九龙油麻地，1952年随双亲定居台湾台北市。'
sequence_zh3='馬英九，台灣政治人物、學者，曾任第12、13任中華民國總統、第4、6任中國國民黨主席等職。祖籍江西省永新縣，籍貫湖南省湘潭縣，馬鶴凌和秦厚修之子，生於英屬香港九龍油麻地，1952年隨雙親定居臺灣臺北市。'

## **方法一**

使用class pipeline.<br>
簡潔版.<br>

In [3]:
from transformers import pipeline

#英文
ner_pipe1 = pipeline("ner")
clear_output()
for entity in ner_pipe1(sequence_en1):
    print(entity)

{'entity': 'I-ORG', 'score': 0.99957865, 'index': 1, 'word': 'Hu', 'start': 0, 'end': 2}
{'entity': 'I-ORG', 'score': 0.9909764, 'index': 2, 'word': '##gging', 'start': 2, 'end': 7}
{'entity': 'I-ORG', 'score': 0.9982224, 'index': 3, 'word': 'Face', 'start': 8, 'end': 12}
{'entity': 'I-ORG', 'score': 0.9994879, 'index': 4, 'word': 'Inc', 'start': 13, 'end': 16}
{'entity': 'I-LOC', 'score': 0.9994344, 'index': 11, 'word': 'New', 'start': 40, 'end': 43}
{'entity': 'I-LOC', 'score': 0.99931955, 'index': 12, 'word': 'York', 'start': 44, 'end': 48}
{'entity': 'I-LOC', 'score': 0.9993794, 'index': 13, 'word': 'City', 'start': 49, 'end': 53}
{'entity': 'I-LOC', 'score': 0.98625815, 'index': 19, 'word': 'D', 'start': 79, 'end': 80}
{'entity': 'I-LOC', 'score': 0.95142686, 'index': 20, 'word': '##UM', 'start': 80, 'end': 82}
{'entity': 'I-LOC', 'score': 0.9336589, 'index': 21, 'word': '##BO', 'start': 82, 'end': 84}
{'entity': 'I-LOC', 'score': 0.9761654, 'index': 28, 'word': 'Manhattan', 'star

In [4]:
#中文
print('-'*30)
ner_pipe2 = pipeline("ner",model="Davlan/bert-base-multilingual-cased-ner-hrl",tokenizer="Davlan/bert-base-multilingual-cased-ner-hrl")
clear_output()
for entity in ner_pipe2(sequence_zh3):
    print(entity)

{'entity': 'B-PER', 'score': 0.9997855, 'index': 1, 'word': '馬', 'start': 0, 'end': 1}
{'entity': 'I-PER', 'score': 0.9997669, 'index': 2, 'word': '英', 'start': 1, 'end': 2}
{'entity': 'I-PER', 'score': 0.9997919, 'index': 3, 'word': '九', 'start': 2, 'end': 3}
{'entity': 'B-LOC', 'score': 0.9999001, 'index': 5, 'word': '台', 'start': 4, 'end': 5}
{'entity': 'I-LOC', 'score': 0.9998473, 'index': 6, 'word': '灣', 'start': 5, 'end': 6}
{'entity': 'B-LOC', 'score': 0.9999087, 'index': 22, 'word': '中', 'start': 23, 'end': 24}
{'entity': 'I-LOC', 'score': 0.99985325, 'index': 23, 'word': '華', 'start': 24, 'end': 25}
{'entity': 'I-LOC', 'score': 0.99987614, 'index': 24, 'word': '民', 'start': 25, 'end': 26}
{'entity': 'I-LOC', 'score': 0.9998759, 'index': 25, 'word': '國', 'start': 26, 'end': 27}
{'entity': 'B-ORG', 'score': 0.99976295, 'index': 34, 'word': '中', 'start': 35, 'end': 36}
{'entity': 'I-ORG', 'score': 0.99979717, 'index': 35, 'word': '國', 'start': 36, 'end': 37}
{'entity': 'I-ORG', '

## **方法二**

詳細版.<br>
方法二的最後輸出格式和方法一的不太一樣.<br>

此處比對三個不同中文文本的輸入.<br>

In [5]:
from transformers import TFAutoModelForTokenClassification, AutoTokenizer
import tensorflow as tf

#英文
# model = TFAutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#中文
model = TFAutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
clear_output()
#tokenizer1 = AutoTokenizer.from_pretrained("bert-base-chinese") #用這個tokenizer產生的輸出也是可行的.


#sequence_zh1
inputs1 = tokenizer(sequence_zh1, return_tensors="tf")
tokens = inputs1.tokens()

outputs = model(inputs1)[0]
predictions = tf.argmax(outputs, axis=2)


for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

('[CLS]', 'O')
('我', 'O')
('们', 'O')
('藏', 'O')
('有', 'O')
('一', 'O')
('册', 'O')
('１', 'O')
('##９', 'O')
('##４', 'O')
('##５', 'O')
('年', 'O')
('６', 'O')
('月', 'O')
('油', 'O')
('印', 'O')
('的', 'O')
('《', 'O')
('北', 'B-LOC')
('京', 'I-LOC')
('文', 'O')
('物', 'O')
('保', 'O')
('存', 'O')
('保', 'O')
('管', 'O')
('状', 'O')
('态', 'O')
('之', 'O')
('调', 'O')
('查', 'O')
('报', 'O')
('告', 'O')
('》', 'O')
('，', 'O')
('调', 'O')
('查', 'O')
('范', 'O')
('围', 'O')
('涉', 'O')
('及', 'O')
('故', 'B-LOC')
('宫', 'I-LOC')
('、', 'O')
('历', 'B-LOC')
('博', 'I-LOC')
('、', 'O')
('古', 'B-ORG')
('研', 'I-ORG')
('所', 'I-ORG')
('、', 'O')
('北', 'B-LOC')
('大', 'I-LOC')
('清', 'I-LOC')
('华', 'I-LOC')
('图', 'I-LOC')
('书', 'I-LOC')
('馆', 'I-LOC')
('、', 'O')
('北', 'B-LOC')
('图', 'I-LOC')
('、', 'O')
('日', 'B-LOC')
('伪', 'O')
('资', 'O')
('料', 'O')
('库', 'O')
('等', 'O')
('二', 'O')
('十', 'O')
('几', 'O')
('家', 'O')
('，', 'O')
('言', 'O')
('及', 'O')
('文', 'O')
('物', 'O')
('二', 'O')
('十', 'O')
('万', 'O')
('件', 'O')
('以', 'O')
('上', 'O')
(

In [6]:
#sequence_zh2

inputs2 = tokenizer(sequence_zh2, return_tensors="tf")
tokens = inputs2.tokens()

outputs = model(inputs2)[0]
predictions = tf.argmax(outputs, axis=2)


for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

('[CLS]', 'O')
('马', 'B-PER')
('英', 'I-PER')
('九', 'I-PER')
('，', 'O')
('台', 'B-LOC')
('湾', 'I-LOC')
('政', 'O')
('治', 'O')
('人', 'O')
('物', 'O')
('、', 'O')
('学', 'O')
('者', 'O')
('，', 'O')
('曾', 'O')
('任', 'O')
('第', 'O')
('12', 'O')
('、', 'O')
('13', 'O')
('任', 'O')
('中', 'B-LOC')
('华', 'I-LOC')
('民', 'I-LOC')
('国', 'I-LOC')
('总', 'O')
('统', 'O')
('、', 'O')
('第', 'O')
('4', 'O')
('、', 'O')
('6', 'O')
('任', 'O')
('中', 'B-ORG')
('国', 'I-ORG')
('国', 'I-ORG')
('民', 'I-ORG')
('党', 'I-ORG')
('主', 'O')
('席', 'O')
('等', 'O')
('职', 'O')
('。', 'O')
('祖', 'O')
('籍', 'O')
('江', 'B-LOC')
('西', 'I-LOC')
('省', 'I-LOC')
('永', 'B-LOC')
('新', 'I-LOC')
('县', 'I-LOC')
('，', 'O')
('籍', 'O')
('贯', 'O')
('湖', 'B-LOC')
('南', 'I-LOC')
('省', 'I-LOC')
('湘', 'B-LOC')
('潭', 'I-LOC')
('县', 'I-LOC')
('，', 'O')
('马', 'B-PER')
('鹤', 'I-PER')
('凌', 'I-PER')
('和', 'O')
('秦', 'B-PER')
('厚', 'I-PER')
('修', 'I-PER')
('之', 'O')
('子', 'O')
('，', 'O')
('生', 'O')
('于', 'O')
('英', 'O')
('属', 'O')
('香', 'B-LOC')
('港', 'I-LOC')


In [7]:
#sequence_zh3

inputs3 = tokenizer(sequence_zh3, return_tensors="tf")
tokens = inputs3.tokens()

outputs = model(inputs3)[0]
predictions = tf.argmax(outputs, axis=2)


for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

('[CLS]', 'O')
('馬', 'B-PER')
('英', 'I-PER')
('九', 'I-PER')
('，', 'O')
('台', 'B-LOC')
('灣', 'I-LOC')
('政', 'O')
('治', 'O')
('人', 'O')
('物', 'O')
('、', 'O')
('學', 'O')
('者', 'O')
('，', 'O')
('曾', 'O')
('任', 'O')
('第', 'O')
('12', 'O')
('、', 'O')
('13', 'O')
('任', 'O')
('中', 'B-LOC')
('華', 'I-LOC')
('民', 'I-LOC')
('國', 'I-LOC')
('總', 'O')
('統', 'O')
('、', 'O')
('第', 'O')
('4', 'O')
('、', 'O')
('6', 'O')
('任', 'O')
('中', 'B-ORG')
('國', 'I-ORG')
('國', 'I-ORG')
('民', 'I-ORG')
('黨', 'I-ORG')
('主', 'O')
('席', 'O')
('等', 'O')
('職', 'O')
('。', 'O')
('祖', 'O')
('籍', 'O')
('江', 'B-LOC')
('西', 'I-LOC')
('省', 'I-LOC')
('永', 'B-LOC')
('新', 'I-LOC')
('縣', 'I-LOC')
('，', 'O')
('籍', 'O')
('貫', 'O')
('湖', 'B-LOC')
('南', 'I-LOC')
('省', 'I-LOC')
('湘', 'B-LOC')
('潭', 'I-LOC')
('縣', 'I-LOC')
('，', 'O')
('馬', 'B-PER')
('鶴', 'I-PER')
('凌', 'I-PER')
('和', 'O')
('秦', 'B-PER')
('厚', 'I-PER')
('修', 'I-PER')
('之', 'O')
('子', 'O')
('，', 'O')
('生', 'O')
('於', 'O')
('英', 'O')
('屬', 'O')
('香', 'B-LOC')
('港', 'I-LOC')


In [8]:
#比對token和id以及原始文字

print('-'*30)
print('sequence_zh1:')
print(inputs1['input_ids'])
print(tokenizer.convert_ids_to_tokens(inputs1['input_ids'][0]))
print('-'*30)
print('sequence_zh2:')
print(inputs2['input_ids'])
print(tokenizer.convert_ids_to_tokens(inputs2['input_ids'][0]))
print('-'*30)
print('sequence_zh3:')
print(inputs3['input_ids'])
print(tokenizer.convert_ids_to_tokens(inputs3['input_ids'][0]))


# #
# print('-'*30)
# print('sequence_zh1:')
# print(inputs1['input_ids'])
# print(tokenizer.convert_ids_to_tokens(inputs1['input_ids'][0]))
# print('-'*30)
# print('sequence_zh2:')
# print(inputs2)
# print('-'*30)
# print('sequence_zh3:')
# print(inputs3)


------------------------------
sequence_zh1:
tf.Tensor(
[[   101   3976   2206   6917   4461   2072   2474  10069 119520 119515
  119516   3642  10074   4460   4929   2695   5718   1888   2650   2172
    4313   5407   2312   3355   2312   6098   5422   3813   2120   7364
    4547   4047   2810   1889  10064   7364   4547   6724   3012   5011
    2730   4286   3402   1881   2707   2684   1881   2747   5832   3999
    1881   2650   3197   5061   2674   3015   2138   8502   1881   2650
    3015   1881   4348   2233   7507   4321   3660   6069   2150   2666
    2513   3408  10064   7168   2730   4313   5407   2150   2666   2075
    2210   2204   2078  10064   4963   4963   2077   2075   2256   7168
   10064   4380   5499   7495   5718   2650   2172   2759   4321   1882
     102]], shape=(1, 101), dtype=int32)
['[CLS]', '我', '们', '藏', '有', '一', '册', '１', '##９', '##４', '##５', '年', '６', '月', '油', '印', '的', '《', '北', '京', '文', '物', '保', '存', '保', '管', '状', '态', '之', '调', '查', '报', '告', '》', '，